<a href="https://colab.research.google.com/github/kanika-sehgal123/Hometask2/blob/main/classification_modeltraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import files
import zipfile

# Upload the data.zip file
uploaded = files.upload()

Saving data.zip to data (6).zip


In [26]:
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [29]:
!ls data
!ls data/Priority
!ls data/Stop


Priority  Stop
0054d8da-0ed5-11ef-88be-a46bb606fa86.jpg  570694be-ed9a-11ed-b020-a46bb6069316.jpg
012168c8-0ed5-11ef-88be-a46bb606fa86.jpg  5723d240-ed9a-11ed-b020-a46bb6069316.jpg
01f363d2-0ed5-11ef-88be-a46bb606fa86.jpg  57cfd84e-0ed5-11ef-88be-a46bb606fa86.jpg
054bc47a-0ed5-11ef-88be-a46bb606fa86.jpg  59c12c14-ed9a-11ed-b020-a46bb6069316.jpg
0693a67c-0ed5-11ef-88be-a46bb606fa86.jpg  59d73f18-ed9a-11ed-b020-a46bb6069316.jpg
0729d336-0ed5-11ef-88be-a46bb606fa86.jpg  59ef2f38-ed9a-11ed-b020-a46bb6069316.jpg
0a394700-0ed5-11ef-88be-a46bb606fa86.jpg  5a0d84c4-ed9a-11ed-b020-a46bb6069316.jpg
0b91ea08-0ed5-11ef-88be-a46bb606fa86.jpg  5a48a784-ed9a-11ed-b020-a46bb6069316.jpg
0cc3e570-0ed5-11ef-88be-a46bb606fa86.jpg  5ad82f0a-0ed5-11ef-88be-a46bb606fa86.jpg
210f1f14-0ed4-11ef-baef-a46bb606fa86.jpg  5b6f18b6-0ed5-11ef-88be-a46bb606fa86.jpg
252b8efc-0ed4-11ef-baef-a46bb606fa86.jpg  5bf29c22-0ed5-11ef-88be-a46bb606fa86.jpg
2607f05e-0ed4-11ef-baef-a46bb606fa86.jpg  5d754ea8-ed9a-11ed-b020-a46bb6

In [30]:
# Install Necessary Libraries
!pip install torch torchvision

# Import Libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
import zipfile
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import copy

# Data Preparation
data_dir = 'data'  # Main directory containing Priority and Stop subdirectories

# Define transforms for training and validation sets
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create a custom dataset loader
def create_datasets(data_dir, transform, val_split=0.2):
    full_dataset = datasets.ImageFolder(data_dir, transform)
    dataset_size = len(full_dataset)
    val_size = int(dataset_size * val_split)
    train_size = dataset_size - val_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    return train_dataset, val_dataset

# Create datasets
train_dataset, val_dataset = create_datasets(data_dir, data_transforms['train'], val_split=0.2)

# Create dataloaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)
}

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}

class_names = train_dataset.dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the Model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

# Train ResNet18
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, len(class_names))
resnet18 = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_resnet, step_size=7, gamma=0.1)

resnet18 = train_model(resnet18, criterion, optimizer_resnet, exp_lr_scheduler, num_epochs=25)

# Train MobileNetV3
mobilenet_v3 = models.mobilenet_v3_large(pretrained=True)
num_ftrs = mobilenet_v3.classifier[3].in_features
mobilenet_v3.classifier[3] = nn.Linear(num_ftrs, len(class_names))
mobilenet_v3 = mobilenet_v3.to(device)

optimizer_mobilenet = optim.SGD(mobilenet_v3.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_mobilenet, step_size=7, gamma=0.1)

mobilenet_v3 = train_model(mobilenet_v3, criterion, optimizer_mobilenet, exp_lr_scheduler, num_epochs=25)

# Save the Models
torch.save(resnet18.state_dict(), 'resnet18_stop_priority.pth')
torch.save(mobilenet_v3.state_dict(), 'mobilenet_v3_stop_priority.pth')


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can als

Epoch 0/24
----------


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


train Loss: 0.5781 Acc: 0.7195
val Loss: 0.4499 Acc: 0.8533

Epoch 1/24
----------
train Loss: 0.3094 Acc: 0.9241
val Loss: 0.2107 Acc: 0.9067

Epoch 2/24
----------
train Loss: 0.1567 Acc: 0.9604
val Loss: 0.1727 Acc: 0.8933

Epoch 3/24
----------
train Loss: 0.1018 Acc: 0.9736
val Loss: 0.1214 Acc: 0.9333

Epoch 4/24
----------
train Loss: 0.0639 Acc: 0.9967
val Loss: 0.0673 Acc: 1.0000

Epoch 5/24
----------
train Loss: 0.0477 Acc: 0.9967
val Loss: 0.0623 Acc: 0.9733

Epoch 6/24
----------
train Loss: 0.0291 Acc: 1.0000
val Loss: 0.0408 Acc: 1.0000

Epoch 7/24
----------
train Loss: 0.0314 Acc: 1.0000
val Loss: 0.0458 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.0290 Acc: 0.9967
val Loss: 0.0434 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.0322 Acc: 1.0000
val Loss: 0.0472 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.0324 Acc: 0.9967
val Loss: 0.0517 Acc: 0.9867

Epoch 11/24
----------
train Loss: 0.0265 Acc: 0.9967
val Loss: 0.0460 Acc: 1.0000

Epoch 12/24
----------
t

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 65.8MB/s]


Epoch 0/24
----------


KeyboardInterrupt: 

# New Section